# Search through gnps libraries to select dataset

Starting from gnps public libraries (speclibs folders, version from 5 July 2019), we here create one large MGF file with all spectra.
In addition Smiles or Inchi's are added from metadata file.

In [1]:
# Locations
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_Wageningen_iOMEGA"
PATH_MS_DATA = ROOT + "\\Data\\GNPS_all\\"
PATH_SAVE_MODEL = ROOT + "\\Spec2Vec\\models_trained\\"
PATH_SAVE_DATA = ROOT + "\\Spec2Vec\\data\\"
PATH_SPEC2VEC = ROOT + "\\Spec2Vec\\code\\"

In [2]:
# Import
import json
import os
import sys
sys.path.insert(0, PATH_SPEC2VEC)

import MS_functions #import Spec2Vec functions related to MS spectra

## Importing additional metadata

In [3]:
json_file = PATH_MS_DATA + 'gnpslibraryjson_filtered.json'

with open(json_file, 'rb') as f:
    data = json.load(f)

In [4]:
print("Metadata available for", len(data), "spectra.")

Metadata available for 77253 spectra.


In [5]:
gnps_data = {}
for data_entry in data:
    if data_entry['GNPSID'] in gnps_data:
        print('duplicate!')
    gnps_data[data_entry['GNPSID']] = data_entry

## Briefly inspect metadata

In [6]:
keys = []
for key in gnps_data.keys():
    keys.append(key)

In [7]:
keys[0]

'CCMSLIB00000001548'

In [8]:
gnps_data[keys[0]]

{'COMPOUND_INCHIKEY': 'KNGPFNUOXXLKCN-ZNCJFREWSA-N',
 'COMPOUND_INCHI': 'InChI=1S/C45H73N5O10S3/c1-14-17-24(6)34(52)26(8)37-25(7)30(58-13)18-31-46-29(19-61-31)39-49-45(12,21-62-39)43-50-44(11,20-63-43)42(57)48-32(22(4)15-2)35(53)27(9)40(55)59-36(23(5)16-3)38(54)47-33(28(10)51)41(56)60-37/h19,22-28,30,32-37,51-53H,14-18,20-21H2,1-13H3,(H,47,54)(H,48,57)/t22-,23-,24+,25-,26-,27+,28+,30-,32-,33-,34-,35-,36-,37-,44+,45+/m0/s1',
 'LIBRARY_QUALITY': '1',
 'GNPSID': 'CCMSLIB00000001548',
 'COMPOUND_NAME': 'Hoiamide B',
 'COMPOUND_SMILES': 'CCC[C@@H](C)[C@@H]([C@H](C)[C@@H]1[C@H]([C@H](Cc2nc(cs2)C3=N[C@](CS3)(C4=N[C@](CS4)(C(=O)N[C@H]([C@H]([C@H](C(=O)O[C@H](C(=O)N[C@H](C(=O)O1)[C@@H](C)O)[C@@H](C)CC)C)O)[C@@H](C)CC)C)C)OC)C)O'}

## Functions

In [9]:
def check_spectrum_lines(spectrum):
    """ Check if desired metadata is present for spectrum.
    """
    spec_id = None
    smiles = None
    
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spec_id = l.strip().split('=')[1]
        if l.startswith('SMILES'):
            smiles = l.strip().split('=')[1]
    
    if spec_id in gnps_data:
        return True
    else:
        return False
    if smiles is not None \
        and smiles.strip() != 'N/A' \
        and smiles.strip() != 'NA'\
        and len(smiles) > 0 \
        and smiles.strip() != 'InChI':
        return True

    return False

def augment_spectrum(spectrum):
    """ Add metadata to spectrum (taken from json file, see above).
    """
    for l in spectrum:
        if l.startswith('SPECTRUMID'):
            spectrum_id = l.strip().split('=')[1]
                
    new_spectrum = []
    for l in spectrum:
        if l.startswith('BEGIN ION'):
            new_spectrum.append(l)
            new_spectrum.append('TITLE=%s\n' % gnps_data[spectrum_id]['COMPOUND_NAME'])
            new_spectrum.append('LIBRARYQUALITY=%s\n' % gnps_data[spectrum_id]['LIBRARY_QUALITY'])
            new_spectrum.append('SMILES=%s\n' % gnps_data[spectrum_id]['COMPOUND_SMILES'])
            new_spectrum.append('INCHIKEY=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHIKEY'])
            new_spectrum.append('INCHI=%s\n' % gnps_data[spectrum_id]['COMPOUND_INCHI'])
            continue
            
        if l.startswith('INCHIKEY'):
            continue
        elif l.startswith('SMILES'):
            continue
        elif l.startswith('INCHI'):
            continue
        elif l.startswith('LIBRARYQUALITY'):
            continue
        elif l.startswith('TITLE'):
            continue
            
        new_spectrum.append(l)

    return new_spectrum

def match_spectra_in_file(filename, matching=True):
    """ Collect spectra in file (line by line). Check if matching metadata is available.
    
    matching: bool
        If matching = True than only include spectra for which desired metadata is present.
        For matching = False, include all spectra.
    """
    matched_spectra = []
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('BEGIN ION'):
                # Fresh spectrum
                spectrum = []
            spectrum.append(line)
            
            if line.startswith('END ION'):
                
                if check_spectrum_lines(spectrum): # Check if we have structural info
                    try:
                        spectrum = augment_spectrum(spectrum)
                    except KeyError as e:
                        pass
                    matched_spectra.append(spectrum)
                else: 
                    if not matching: # then include all spectra
                        matched_spectra.append(spectrum)
    return matched_spectra

## Create MGF for all spectra (also non matched ones)

In [10]:
path = PATH_MS_DATA + 'speclibs\\'
collected_spectra = []
for d in os.listdir(path):
    for f in os.listdir(os.path.join(path, d)):
        if not f.endswith('.mgf'):
            continue
        sp = match_spectra_in_file(os.path.join(path, d, f), matching=False)
        collected_spectra.extend(sp)
        print(f, len(collected_spectra))

BILELIB19.mgf 177
CASMI.mgf 745
DEREPLICATOR_IDENTIFIED_LIBRARY.mgf 1124
GNPS-COLLECTIONS-MISC.mgf 1170
GNPS-COLLECTIONS-PESTICIDES-NEGATIVE.mgf 1246
GNPS-COLLECTIONS-PESTICIDES-POSITIVE.mgf 1899
GNPS-EMBL-MCF.mgf 2484
GNPS-FAULKNERLEGACY.mgf 2611
GNPS-LIBRARY.mgf 7394
GNPS-NIH-CLINICALCOLLECTION1.mgf 7771
GNPS-NIH-CLINICALCOLLECTION2.mgf 7966
GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf 9233
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE.mgf 11096
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE.mgf 16892
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE.mgf 18352
GNPS-NIST14-MATCHES.mgf 24115
GNPS-PRESTWICKPHYTOCHEM.mgf 24258
GNPS-SELLECKCHEM-FDA-PART1.mgf 26646
GNPS-SELLECKCHEM-FDA-PART2.mgf 27302
HMDB.mgf 29537
LDB_NEGATIVE.mgf 29763
LDB_POSITIVE.mgf 29846
MASSBANK.mgf 41845
MASSBANKEU.mgf 43337
MIADB.mgf 43509
MMV_NEGATIVE.mgf 43556
MMV_POSITIVE.mgf 43666
MONA.mgf 92907
PNNL-LIPIDS-NEGATIVE.mgf 109049
PNNL-LIPIDS-POSITIVE.mgf 139631
RESPECT.mgf 146743
SUMNER.mgf 147647


In [11]:
print("Collected", len(collected_spectra), "spectra from: \n", path)

Collected 147647 spectra from: 
 C:\OneDrive - Netherlands eScience Center\Project_Wageningen_iOMEGA\Data\GNPS_all\speclibs\


# Convert smiles into inchi where necessary

### Make inchi strings more homogeneous in style

In [29]:
def cleanup_inchi(spectrum, rescue_smiles = True):
    """
    Read spectrum, look for inchi. Then:
    1) Make line with inchi homogeneously looking like: 'INCHI="InChI=..."'
    2) if rescue_smiles is True then try to detect inchi that are actually smiles 
    and convert to proper inchi (using openbabel based function from MS_functions.py).
    """
  
    spectrum_new = []
    inchi = 'N/A'

    for l in spectrum:
        if l.startswith('INCHI='):
            inchi = l.strip().split('INCHI=')[1]
            
            if inchi is None \
                or inchi in ['N/A', 'NA', 0, '0', '""'] \
                or len(inchi) == 0:
                pass
            else:
                inchi = inchi.replace(" ", "") #remove empty spaces
                if inchi.split('InChI=')[-1][0] in ['C', 'c', 'O', 'N']:
                    if rescue_smiles:
                        # Try to 'rescue' given inchi which are actually smiles!
                        assumed_smile = inchi.split('InChI=')[-1].replace('"', '')
                        inchi = MS_functions.mol_converter(assumed_smile, "smi", "inchi")
                        print('new inchi:', inchi, assumed_smile)
                
                inchi = inchi.strip().split('InChI=')[-1]
                if inchi.endswith('"'):
                    inchi = '"InChI=' + inchi
                elif inchi.endswith('\n'):
                    inchi = '"InChI=' + inchi[:-2] + '"'
                elif inchi.endswith('\n"'):
                    inchi = '"InChI=' + inchi[:-3] + '"'
                else:
                    inchi = '"InChI=' + inchi + '"'
                l = 'INCHI=' + inchi + '\n'
        spectrum_new.append(l)
    return spectrum_new

In [30]:
for i, spectrum in enumerate(collected_spectra):      
    spectrum_new = cleanup_inchi(spectrum, rescue_smiles = True)
    collected_spectra[i] = spectrum_new 

new inchi: InChI=1S/C7H8N4O2/c1-10-5-4(8-3-9-5)6(12)11(2)7(10)13/h3H,1-2H3,(H,8,9)
 Cn(c1nc[nH]c1c(=O)n2C)c2=O
new inchi: InChI=1S/C42H60N6O11/c1-24(2)13-11-9-7-8-10-12-14-35(52)47(5)32(23-49)40(56)44-25(3)38(54)43-22-36(53)48(6)37-28-16-18-34(51)30(21-28)29-19-27(15-17-33(29)50)20-31(42(58)59)46-39(55)26(4)45-41(37)57/h15-19,21,24-26,31-32,37,49-51H,7-14,20,22-23H2,1-6H3,(H,43,54)(H,44,56)(H,45,57)(H,46,55)(H,58,59)
 CC1C(NC(C(O)=O)CC2=CC(C3=C(O)C=CC(C(N(C(CNC(C(NC(C(N(C(CCCCCCCCC(C)C)=O)C)CO)=O)C)=O)=O)C)C(N1)=O)=C3)=C(O)C=C2)=O
new inchi: InChI=1S/C87H139N15O25/c1-16-18-19-20-21-24-54(103)44-70(109)92-56(31-35-66(88)105)77(113)93-57(32-36-67(89)106)78(114)94-58(33-37-68(90)107)79(115)95-59(34-38-69(91)108)86(122)126-52(12)82(118)101-39-22-25-61(101)84(120)99(13)63(42-47(5)6)80(116)98-73-51(11)125-87(123)64(43-53-27-29-55(124-15)30-28-53)100(14)85(121)62-26-23-40-102(62)83(119)60(41-46(3)4)96-76(112)50(10)74(111)75(48(7)8)127-71(110)45-65(104)72(49(9)17-2)97-81(73)117/h27-30,46-52,54

new inchi: InChI=1S/C11H12N2O2/c12-9(11(14)15)5-7-6-13-10-4-2-1-3-8(7)10/h1-4,6,9,13H,5,12H2,(H,14,15)/t9-/m0/s1
 O=C(O)[C@@H](N)Cc1c[nH]c2c1cccc2


#### Insert save point here

In [31]:
"""
# Save data here
import pickle
filename = 'collected_spectra_lst'

with open(filename, "wb") as output:
    pickle.dump(collected_spectra, output, pickle.HIGHEST_PROTOCOL);
"""

In [ ]:
"""
import pickle
filename = 'collected_spectra_lst'
collected_spectra = pickle.load(open(filename, "rb"))
"""

### Search for spectra with smiles but no inchi (and try to convert smiles to inchi)

In [32]:
from openbabel import openbabel as ob
from openbabel import pybel

inchi_lst = []
inchikey_lst = []
smiles_lst = []

changes_to_spectra = []
for i, spectrum in enumerate(collected_spectra):
    # Show progress
    if i // 10000 == i/10000:
        print("running spectra no", i)
        
    smiles = 'N/A'
    inchi = 'N/A'
    inchikey = 'N/A'
    modified = False
    
    smiles_line = 0
    inchi_line = 0
    inchikey_line = 0
    inchiaux_line = 0
    for m, l in enumerate(spectrum):
        if l.startswith('SMILES'):
            smiles = l.strip().split('SMILES=')[1]
            smiles_line = 0
        if l.startswith('INCHI='):
            inchi = l.strip().split('INCHI=')[1]
            inchi_line = m
        if l.startswith('INCHIKEY='):
            inchikey = l.strip().split('INCHIKEY=')[1]
            inchikey_line = m
        if l.startswith('INCHIAUX='):
            inchiaux_line = m

    # Add inchi line if not present        
    if inchi_line == 0: 
        spectrum.insert(smiles_line+1, 'INCHI=N/A\n')

    # Add inchikey line if not present
    if inchikey_line == 0: 
        if inchiaux_line == 0:
            spectrum.insert(inchi_line+1, 'INCHIKEY=N/A\n')
        else:
            spectrum[inchiaux_line] = 'INCHIKEY=N/A\n'           
        
    # Add to lists
    smiles_lst.append(smiles)
    inchi_lst.append(inchi)
    inchikey_lst.append(inchikey)
    
    # Look for cases with smiles BUT NO inchi 
    if inchi is None \
        or inchi in ['N/A', 'NA', 0, '0'] \
        or len(inchi) == 0:
            if smiles is not None \
                and smiles not in ['N/A', 'NA', 0, '0', 'InChI'] \
                and len(smiles) > 0:  
                    for m, l in enumerate(spectrum):
                        if l.startswith('INCHI='):
                            inchi = MS_functions.mol_converter(smiles, "smi", "inchi")
                            
                            if inchi.endswith("\n"):
                                inchi = inchi[:-2]
                            l = 'INCHI=' + '"' + inchi+ '"\n'
                            spectrum[m] = l
                            modified = True
                            inchi_lst[-1] = inchi 
                            continue #assuming there is only one line with inchi

    # Look for missing inchikeys
    if inchikey is None \
        or inchikey in ['N/A', 'NA', 0, '0'] \
        or len(inchikey) == 0:
            if len(inchi) > 12:
                if inchi.split('=')[1][0] == '1': # Filter out smiles or defective inchi given as inchi
                    #inchi_to_keys = ob.OBConversion()
                    #inchi_to_keys.SetInAndOutFormats("inchi", "inchikey")
                    #mol = ob.OBMol()
                    #inchi_to_keys.ReadString(mol, inchi)
                    #found_inchikey = inchi_to_keys.WriteString(mol)
                    found_inchikey = MS_functions.mol_converter(inchi, "inchi", "inchikey")

                    for m, l in enumerate(spectrum):
                        if l.startswith('INCHIKEY='):
                            l = 'INCHIKEY=%s\n' % found_inchikey
                            spectrum[m] = l
                            modified = True
                            inchikey_lst[-1] = found_inchikey
                else:
                    print('Suspicious inchi found for spectrum', i, ': ', inchi )

    # Replace spectrum with modified one                                 
    if modified:                                       
        collected_spectra[i] = spectrum                  
        changes_to_spectra.append((i, inchi_lst[-1], inchikey_lst[-1]))

running spectra no 0
Suspicious inchi found for spectrum 5418 :  "InChI=DOUMFZQKYFQNTF-WUTVXBCWSA-N"
Suspicious inchi found for spectrum 5419 :  "InChI=DOUMFZQKYFQNTF-WUTVXBCWSA-N"
Suspicious inchi found for spectrum 5420 :  "InChI=QGJZLNKBHJESQX-FZFNOLFKSA-N"
Suspicious inchi found for spectrum 5421 :  "InChI=WCGUUGGRBIKTOS-GPOJBZKASA-N"
Suspicious inchi found for spectrum 5422 :  "InChI=WCGUUGGRBIKTOS-GPOJBZKASA-N"
Suspicious inchi found for spectrum 7932 :  "InChI=Brc1c(NC2=NCCN2)ccc3nccnc13"
running spectra no 10000
running spectra no 20000
running spectra no 30000
running spectra no 40000
running spectra no 50000
running spectra no 60000
running spectra no 70000
running spectra no 80000
running spectra no 90000
running spectra no 100000
running spectra no 110000
running spectra no 120000
running spectra no 130000
running spectra no 140000


In [33]:
inchikey_lst[:20]

['RUDATBOHQWOJDD-GYHJSIKNSA-N\n',
 'RUDATBOHQWOJDD-PODLAYBOSA-N\n',
 'RUDATBOHQWOJDD-PODLAYBOSA-N\n',
 'BHQCQFFYRZLCQQ-IHELEQLESA-N\n',
 'BHQCQFFYRZLCQQ-IHELEQLESA-N\n',
 'BHQCQFFYRZLCQQ-IHELEQLESA-N\n',
 'KXGVEGMKQFWNSR-BKAWJTANSA-N\n',
 'KXGVEGMKQFWNSR-BKAWJTANSA-N\n',
 'KXGVEGMKQFWNSR-BKAWJTANSA-N\n',
 'GHCZAUBVMUEKKP-AJEXLSFKSA-N\n',
 'GHCZAUBVMUEKKP-AJEXLSFKSA-N\n',
 'GHCZAUBVMUEKKP-AJEXLSFKSA-N\n',
 'RFDAIACWWDREDC-VKRKCYKBSA-N\n',
 'RFDAIACWWDREDC-VKRKCYKBSA-N\n',
 'RFDAIACWWDREDC-VKRKCYKBSA-N\n',
 'WVULKSPCQVQLCU-JKPPYMBGSA-N\n',
 'WVULKSPCQVQLCU-JKPPYMBGSA-N\n',
 'WVULKSPCQVQLCU-JKPPYMBGSA-N\n',
 'ZQYUKJFJPJDMMR-ZDWCHQGWSA-N\n',
 'ZQYUKJFJPJDMMR-ZDWCHQGWSA-N\n']

In [34]:
inchi_lst[:5], smiles_lst[:5]

(['"InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s1"',
  '"InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s2"',
  '"InChI=1S/C24H40O4/c1-14(4-7-21(27)28)17-5-6-18-22-19(9-11-24(17,18)3)23(2)10-8-16(25)12-15(23)13-20(22)26/h14-20,22,25-26H,4-13H2,1-3H3,(H,27,28)/t14?,15-,16+,17+,18-,19-,20+,22-,23?,24?/m0/s2"',
  '"InChI=1S/C24H40O5/c1-13(4-7-21(28)29)16-5-6-17-22-18(12-20(27)24(16,17)3)23(2)9-8-15(25)10-14(23)11-19(22)26/h13-20,22,25-27H,4-12H2,1-3H3,(H,28,29)/t13?,14-,15+,16+,17-,18-,19+,20-,22-,23?,24?/m0/s1"',
  '"InChI=1S/C24H40O5/c1-13(4-7-21(28)29)16-5-6-17-22-18(12-20(27)24(16,17)3)23(2)9-8-15(25)10-14(23)11-19(22)26/h13-20,22,25-27H,4-12H2,1-3H3,(H,28,29)/t13?,14-,15+,16+,17-,18-,19+,20-,22-,23?

In [35]:
file_mgf = PATH_MS_DATA + 'all_gnps_inchi_added_191029.mgf'
with open(file_mgf, 'w') as f:
    for c in collected_spectra[:]:
        for l in c:
            try:
                f.write(l.encode('utf8', 'replace').decode('utf8', 'replace'))
            except UnicodeEncodeError:
                f.write(l.encode('ascii', 'replace').decode('utf8', 'replace'))
                print(l)
        f.write('\n')

TITLE=C16 Galactosyl(�) Ceramide (d18:1/16:0)

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8,19-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})-4,9,15,20-tetraoxatricyclo[16.4.0.0?,��]docosa-6,10,17,21-tetraene-11,22-dicarboxylate

TITLE=11,22-dimethyl (6E,17E)-3,14-dioxo-8

TITLE=9-methoxy-4,4-dimethyl-13-phenyl-6,14-dioxatetracyclo[8.4.0.0�,?.0�,?]tetradeca-1,7,9-trien-11-one

TITLE=9-methoxy-4,4-dimethyl-13-phenyl-6,14-dioxatetracyclo[8.4.0.0�,?.0�,?]tetradeca-1,7,9-trien-11-one

TITLE=9-methoxy-4,4-dimethyl-13-phenyl-6,14-dioxatetracyclo[8.4.0.0�,?.0�,?]tetradeca-1,7,9-trien-11-one

TITLE=9-methoxy-4,4-dimethyl-13-phenyl-6,14-dioxatetracyclo[8.4.0.0�,?.0�,?]tetradeca-1,7,9-trien-11-one

TITLE=11-(hydroxymethyl)-2,6,6,14,19-pentamethyl-7,16,18-trioxapentacyclo[12.6.1.0�,��.0?,�?.0�?,��]henicosa-5(10),11-diene-3,8,15,20-tetrone

TITLE=11-(hydroxymethyl)-2,6,6,14,19-pentamethyl-7,16,18-trioxapentacyclo[12.6.1.0�,��.0?,�?.0�?,��]henicosa-5(10),11-diene-3,8,15,20-tetrone

TITLE=11-(hydroxymethyl)-2,6,6,14,19-pentamethyl-7,16,18-trioxapentacyclo[12.6.1.0�,��.0?,�?.0�?,��]henicosa-5(10),11-diene-3,8,15,20-tetrone

TITLE=11-(hydroxymethyl)-2,6,6,14,19-pentamethyl-7,16,18-trioxapentacyclo[12.6.1.0�,��.0?,�?.0�?,��]henicosa-5(10),11-diene-3,8,15,20-tetrone

TITL

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0?,�?.0?,�?.0?,��]tricosa-6(14),7,12-trien-2-yl acetate

TITLE=(1R,2R,4S,16R,17R,20S)-13,20-dimethoxy-4,7,17,22,22-pentamethyl-11-oxo-5,10,21,23-tetraoxahexacyclo[18.2.1.0�,�?.0

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=8-benzoyl-1,5,5-trimethyl-6,15-dioxatetracyclo[9.3.1.0?,��.0?,��]pentadeca-7(12),8,10-trien-9-ol

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4,5-trihydroxybenzoyloxy)-3,6,9,24,27,33-hexaoxaheptacyclo[28.7.1.0?,�?.0?,�?.0��,�?.0�?,��.0�?,�?]octatriaconta-1(38),11(16),12,14,17,19,21,34,36-nonaen-29-yl]acetic acid

TITLE=2-[(5S,25R,26R,29S,30S,31S)-13,14,15,18,19,20,31,35,36-nonahydroxy-2,10,23,28,32-pentaoxo-5-(3,4

TITLE=2-[(5S,8R,11S,12S,13S,21R)-13,17,18-trihydroxy-2,10,14-trioxo-5,21-bis(3,4,5-trihydroxybenzoyloxy)-7-[(3,4,5-trihydroxybenzoyloxy)methyl]-3,6,9,15-tetraoxatetracyclo[10.7.1.1?,?.0�?,�?]henicosa-1(20),16,18-trien-11-yl]acetic acid

TITLE=2-[(5S,8R,11S,12S,13S,21R)-13,17,18-trihydroxy-2,10,14-trioxo-5,21-bis(3,4,5-trihydroxybenzoyloxy)-7-[(3,4,5-trihydroxybenzoyloxy)methyl]-3,6,9,15-tetraoxatetracyclo[10.7.1.1?,?.0�?,�?]henicosa-1(20),16,18-trien-11-yl]acetic acid

TITLE=2-[(5S,8R,11S,12S,13S,21R)-13,17,18-trihydroxy-2,10,14-trioxo-5,21-bis(3,4,5-trihydroxybenzoyloxy)-7-[(3,4,5-trihydroxybenzoyloxy)methyl]-3,6,9,15-tetraoxatetracyclo[10.7.1.1?,?.0�?,�?]henicosa-1(20),16,18-trien-11-yl]acetic acid

TITLE=2-[(5S,8R,11S,12S,13S,21R)-13,17,18-trihydroxy-2,10,14-trioxo-5,21-bis(3,4,5-trihydroxybenzoyloxy)-7-[(3,4,5-trihydroxybenzoyloxy)methyl]-3,6,9,15-tetraoxatetracyclo[10.7.1.1?,?.0�?,�?]henicosa-1(20),16,18-trien-11-yl]acetic acid

TITLE=2-[(5S,8R,11S,12S,13S,21R)-13,17,18-trihydroxy

...

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethyl-10-oxatetracyclo[7.7.1.0�,?.0��,�?]heptadec-5-ene-4,11-dione

TITLE=(2S,3S,9R,12R,13R,14S,15R,16R,17S)-3,12,13,15,16-pentahydroxy-2,6,14,17-tetramethy

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(1R,4S,5R,9R,10R,13S)-13-{[(2S,3R,4S,5R,6R)-5-hydroxy-6-(hydroxymethyl)-3,4-bis({[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy})oxan-2-yl]oxy}-5,9-dimethyl-14-methylidenetetracyclo[11.2.1.0�,�?.0?,?]hexadecane

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-3-yl)oxy]-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}oxan-3-yl)oxy]-6-methyloxane-3,4,5-triol

TITLE=(2S,3R,4R,5R,6S)-2-[(4-hydroxy-6-{[(4S,6R,7S,8R,9S,13S,16S)-6-hydroxy-7,9,13-trimethyl-6-[(3R)-3-methyl-4-{[(2R,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}butyl]-5-oxapentacyclo[10.8.0.0�,?.0?,?.0��,�?]icosan-16-yl]oxy}-2-(hydroxymethyl)-5-{[3,4,5-trihydroxy-6-(hy

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-oxaheptacyclo[15.10.2.0�,�?.0�,�?.0?,�?.0?,��.0��,�?]nonacosa-3,5(14),11,15,19,21(26),22,24,28-nonaene-7-carboxylate

TITLE=methyl 8,11,15,20,22-pentahydroxy-24-methyl-13,18,27-trioxo-6-ox

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(5S,9R)-14-(hydroxymethyl)-5,9-dimethyltetracyclo[11.2.1.0�,�?.0?,?]hexadecane-5-carboxylic acid

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethylidene-4-hydroxy-15-{[(2S,3R,4S,5S,6R)-3,4,5-trihydroxy-6-(hydroxymethyl)oxan-2-yl]oxy}-2,10,14,20-tetraoxatetracyclo[21.2.2.1�,?.0��,�?]octacosa-1(25),3(28),4,6,12,23,26-heptaene-11,19-dione

TITLE=(15S,16Z,17S)-16-ethyl

## Replace charge values of "-1+" with "-1"

In [36]:
input_file = PATH_MS_DATA + 'all_gnps_inchi_added_191029.mgf'
output_file = open(ROOT + "\\Data\\GNPS_all\\" + "MS_data_allGNPS_191029_corrected.mgf", 'w')

with open(input_file, 'r') as file:
    for line in file:
        output_file.write(line.replace('-1+', '-1'))

In [65]:
output_file.close()